In [1]:
import os
import numpy as np
import time
from typing import Tuple, List, Dict
import tensorflow as tf

from createdataset import *

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
def data(subset='pku',padding=50):
    '''function that creates a dataset -- training, dev, and test
    args: subset: any subset, padding: padding size
    returns: (X_train, y_train), (X_dev, y_dev), (X_test, y_test), info_dev
    '''
    
    type_ = "training"
    print(type_)
    Label_file, Input_file = ChooseDataset(type_, subset)
    A = CreateDataset(Label_file, Input_file, padding, type_, None)
    X_train, y_train, info_train, word_to_index_training = A.DateGen()
    print("X shape: {}\ny shape: {}".format(X_train.shape, y_train.shape))
    print(info_train)
    
    type_ = 'dev'
    Label_file, Input_file = ChooseDataset(type_, subset)
    A = CreateDataset(Label_file, Input_file, padding, type_, word_to_index_training)
    X_dev, y_dev, info_dev, _ = A.DateGen()
    print(type_)
    print("X shape: {}\ny shape: {}".format(X_dev.shape, y_dev.shape))
    print(info_dev)
    
    type_ = 'testing'
    Label_file, Input_file = ChooseDataset(type_, subset)
    A = CreateDataset(Label_file, Input_file, padding, type_, word_to_index_training)
    X_test, y_test, info_test, _ = A.DateGen()
    print(type_)
    print("X shape: {}\ny shape: {}".format(X_test.shape, y_test.shape))
    print(info_dev)
    
    return (X_train, y_train), (X_dev, y_dev), (X_test, y_test), info_dev


(X_train, y_train), (X_dev, y_dev), (X_test, y_test), info_dev = data(subset='pku',padding=10)

training


## Model parameters

In [ ]:
#DEFINE SOME COSTANTS
VOCAB_SIZE = info_dev['VocabSize']
EMBEDDING_SIZE = 32
HIDDEN_SIZE = 256
PADDING_SIZE = info_dev['MAXLEN']
batch_size = 128
epochs = 1

In [ ]:
model_name = time.strftime('%Y-%m-%d_%H:%M:%S_%z')
def create_keras_model(vocab_size, embedding_size, hidden_size, PADDING_SIZE):
    print("Creating KERAS model")
    model = K.models.Sequential()
    model.add(K.layers.Embedding(vocab_size, embedding_size, mask_zero=True, input_length = PADDING_SIZE))
    model.add(K.layers.LSTM(hidden_size, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    model.add(K.layers.TimeDistributed(K.layers.Dense(4, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

    return model

model = create_keras_model(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, PADDING_SIZE)
model.summary()

In [ ]:
cbk = K.callbacks.TensorBoard('../resources/logging/keras_model_'+model_name)
print("\nStarting training...")
K.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [ ]:
csv_logger = K.callbacks.CSVLogger('../resources/logging/keras_model_'+model_name+'.log')
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
          shuffle=True, validation_data=(X_dev, y_dev), callbacks=[cbk, csv_logger]) 
print("Training complete.\n")

In [ ]:
if not os.path.exists('../resources/models'):
    os.mkdir('../resources/models')
weights = os.path.join(rel_path,'model_weights_'+model_name+'.h5')
model_name_save = os.path.join(rel_path,'model_'+model_name+'.h5')
model.save_weights(weights) #saving weights for further analysis
model.save(model_name_save)

In [ ]:
print("\nEvaluating test...")
loss_acc = model.evaluate(X_test, y_test, verbose=3)
print("Test data: loss = %0.6f  accuracy = %0.2f%% " % (loss_acc[0], loss_acc[1]*100))